# Demo for the SMILES processing

In [4]:
########################################################
# All rights reserved. 
# Author: XIE Zhengwei @ Beijing Gigaceuticals Tech Co., Ltd 
#                      @ Peking University International Cancer Institute
# Contact: xiezhengwei@gmail.com
#
#
########################################################

import sys
sys.path.append('../DLEPS')
import molecule_vae
from rdkit.Chem import MolFromSmiles, MolToSmiles
from rdkit.Chem import Draw

import numpy as np  
import pandas as pd


In [20]:
dt1 = pd.read_csv('../../data/train_SMILES_demo.csv')
dt2 = pd.read_csv('../../data/test_SMILES_demo.csv')

smiles = np.concatenate([dt1['SMILES'].values, dt2['1'].values],axis=0) 
    

In [21]:
smiles

array(['NC(=O)NCC#Cc1ccc2NC(=O)[C@@]3([C@H]([C@H]4N([C@H]3c5ccc(OCCO)cc5)[C@@H]([C@@H](OC4=O)c6ccccc6)c7ccccc7)C(=O)NCC=C)c2c1',
       'COc1cccc(NC(=O)N2[C@H]3CNC[C@@H]2[C@@H]3c2ccc(cc2)-c2ccncc2)c1',
       'COc1ccc2c3c([C@H](CO)N(C[C@]33CCN(Cc4nccs4)CC3)C(=O)NC(C)C)n(C)c2c1',
       ...,
       'OC[C@@H]1O[C@@H](CC(=O)NCCN2CCCCC2)C[C@H]2[C@@H]1Oc1ccc(NC(=O)c3ccc4OCOc4c3)cc21',
       'CN(C)CCCNc1c2ccccc2n(C)c2nc(=O)n(C)c(=O)c12',
       'COc1cccc(c1)C(=O)N1C[C@@H]2[C@@H]([C@H](CO)N2C(=O)c2ccc(F)cc2)c2ccccc12'],
      dtype=object)

In [42]:
#list(smiles)

In [24]:
print(smiles[2])
MolToSmiles(MolFromSmiles(smiles[2]))

COc1ccc2c3c([C@H](CO)N(C[C@]33CCN(Cc4nccs4)CC3)C(=O)NC(C)C)n(C)c2c1


'COc1ccc2c3c(n(C)c2c1)[C@H](CO)N(C(=O)NC(C)C)CC31CCN(Cc2nccs2)CC1'

In [25]:
#smiles = smiles[:100]
smiles_rdkit = []
iid = []
for i in range(len(smiles)):
    try:
        smiles_rdkit.append(MolToSmiles(MolFromSmiles(smiles[ i ])))
        iid.append(i)
        #print(i)
    except:
        print("Error at %d" % (i))
        

In [26]:
print(len(smiles))
print(len(iid))

1400
1400


In [43]:
#smiles_rdkit

In [32]:
def get_zinc_tokenizer(cfg):
    long_tokens = [a for a in list(cfg._lexical_index.keys()) if xlength(a) > 1] ####
    replacements = ['$','%','^'] # ,'&']
    assert xlength(long_tokens) == len(replacements) ####xzw
    for token in replacements: 
        assert token not in cfg._lexical_index ####
    
    def tokenize(smiles):
        for i, token in enumerate(long_tokens):
            smiles = smiles.replace(token, replacements[i])
        tokens = []
        for token in smiles:
            try:
                ix = replacements.index(token)
                tokens.append(long_tokens[ix])
            except:
                tokens.append(token)
        return tokens
    
    return tokenize


In [34]:
from functools import reduce

def xlength(y):
    return reduce(lambda sum, element: sum + 1, y, 0)


In [35]:
import zinc_grammar
import nltk

_tokenize = get_zinc_tokenizer(zinc_grammar.GCFG)
_parser = nltk.ChartParser(zinc_grammar.GCFG)
_productions = zinc_grammar.GCFG.productions()
_prod_map = {}
for ix, prod in enumerate(_productions):
    _prod_map[prod] = ix
MAX_LEN = 277
_n_chars = len(_productions)


In [37]:
smiles_rd = smiles_rdkit[:100]

In [40]:
        """ Encode a list of smiles strings into the latent space """
        assert type(smiles_rdkit) == list
        tokens = map(_tokenize, smiles_rdkit)
        parse_trees = []
        i = 0
        badi = []
        for t in tokens:
            #while True:
            try:
                tp = next(_parser.parse(t))
                parse_trees.append(tp)
            except:
                print("Parse tree error at %d" % i)
                badi.append(i)
            i += 1
            #print(i)
        productions_seq = [tree.productions() for tree in parse_trees]
        indices = [np.array([_prod_map[prod] for prod in entry], dtype=int) for entry in productions_seq]
        one_hot = np.zeros((len(indices), MAX_LEN, _n_chars), dtype=np.float32)
        for i in range(len(indices)):
            num_productions = len(indices[i])
            if num_productions > MAX_LEN:
                print("Too large molecules, out of range")
            #print("i=  {%d} len(indices)=  {%d} num_productions = %d " % (i,len(indices),num_productions))
                one_hot[i][np.arange(MAX_LEN),indices[i][:MAX_LEN]] = 1.
            else:    
                one_hot[i][np.arange(num_productions),indices[i]] = 1.
                one_hot[i][np.arange(num_productions, MAX_LEN),-1] = 1.

Too large molecules, out of range
Too large molecules, out of range
Too large molecules, out of range
Too large molecules, out of range
Too large molecules, out of range
Too large molecules, out of range
Too large molecules, out of range
Too large molecules, out of range
Too large molecules, out of range
Too large molecules, out of range
Too large molecules, out of range
Too large molecules, out of range
Too large molecules, out of range
Too large molecules, out of range
Too large molecules, out of range
Too large molecules, out of range
Too large molecules, out of range
Too large molecules, out of range
Too large molecules, out of range
Too large molecules, out of range
Too large molecules, out of range
Too large molecules, out of range
Too large molecules, out of range
Too large molecules, out of range
Too large molecules, out of range
Too large molecules, out of range


In [41]:
iix = np.arange(len(smiles_rdkit))
iidx = [i for i in iix if i not in badi]
len(iidx)
iid2 = np.array(iid)[iidx]
len(iid2)

1400

In [15]:
#L962 = np.genfromtxt('L1000.csv', delimiter=',')
#L962 = L962[iid2]
#print(L962.shape)
#num_examples = L962.shape[0]

(17051, 978)


In [16]:
perm = np.arange(num_examples)
np.random.shuffle(perm)
#L962 = L962[perm]
smile_rd = one_hot[perm]
TEST_SIZE = 3000
#L962_train = L962[TEST_SIZE:]
smile_train = smile_rd[TEST_SIZE:]
#L962_test = L962[:TEST_SIZE]
smile_test = smile_rd[:TEST_SIZE]

In [17]:
#savetxt(fname, X, fmt='%.18e', delimiter=' ', newline='\n', header='', footer='', comments='# ', encoding=None)[source]
#np.savetxt("L1000_train.csv",L962_train, delimiter = ',')
#np.savetxt("L1000_test.csv",L962_test, delimiter = ',')

In [18]:
#import h5py

#h5f = h5py.File('SMILE_train_demo.h5', 'w')
#h5f.create_dataset('data', data=smile_train)
#h5f.close()

#h5f = h5py.File('SMILE_test_demo.h5', 'w')
#h5f.create_dataset('data', data=smile_test)
#h5f.close()

